In [4]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')


In [11]:
df = pd.read_csv(r"C:\Users\WANGYILIN\Desktop\ALL_lesson\ML_foundation\22201960.csv",encoding="latin-1")
train_data = pd.read_csv(r"C:\Users\WANGYILIN\Desktop\ALL_lesson\ML foundation\train.csv",encoding="latin-1")






In [10]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# 分词并对文本进行编码
tokenized = train_data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

# 找到最大长度
max_len = max(map(len, tokenized))

# Padding
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# 创建attention_mask
attention_mask = np.where(padded != 0, 1, 0)

# 将padded和attention_mask转化为torch.Tensor
input_ids = torch.LongTensor(padded)
attention_mask = torch.LongTensor(attention_mask)

# 提取特征
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

# 提取[CLS]对应的向量
features = last_hidden_states[0][:,0,:].numpy()

# 准备标签
labels = train_data['category']

# 划分训练集和测试集
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

# 设置待搜索的参数范围
parameters = {'C': np.linspace(0.0001, 100, 20)}

# 进行网格搜索以找到最佳参数
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

# 打印最佳参数和最佳分数
print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

# 使用找到的最佳参数训练模型
lr_clf = LogisticRegression(C = grid_search.best_params_['C'])
lr_clf.fit(train_features, train_labels)

# 测试模型
print("Test Accuracy: ", lr_clf.score(test_features, test_labels))

best parameters:  {'C': 15.789557894736841}
best scrores:  0.9811746392958168
Test Accuracy:  0.9797809604043808


In [19]:
# 读取测试数据
test_data = pd.read_csv(r"C:\Users\WANGYILIN\Desktop\ALL_lesson\ML foundation\test.csv",encoding="latin-1")

print(test_data.isnull().sum())
test_data = test_data.fillna("")
print(test_data.isnull().sum())

test_data['text'] = test_data['headline'] + " " + test_data['short_description']

# 对测试数据进行分词并编码
tokenized_test = test_data['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

# 找到最大长度，注意这里应与训练数据中的最大长度保持一致
max_len_test = max(map(len, tokenized_test))

# 对测试数据进行Padding
padded_test = np.array([i + [0]*(max_len_test-len(i)) for i in tokenized_test.values])

# 创建attention_mask
attention_mask_test = np.where(padded_test != 0, 1, 0)

# 将padded和attention_mask转化为torch.Tensor
input_ids_test = torch.LongTensor(padded_test)
attention_mask_test = torch.LongTensor(attention_mask_test)

# 提取特征
with torch.no_grad():
    last_hidden_states_test = model(input_ids_test, attention_mask=attention_mask_test)

# 提取[CLS]对应的向量作为特征
features_test = last_hidden_states_test[0][:,0,:].numpy()

# 使用训练好的模型进行预测
test_pred = lr_clf.predict(features_test)

# 打印预测结果
print(test_pred)
# 使用训练好的模型进行预测
from sklearn.metrics import classification_report
# 进行评估
y_test = test_data['category']

print(classification_report(y_test, test_pred))

Unnamed: 0           0
headline             1
short_description    0
date                 0
category             0
dtype: int64
Unnamed: 0           0
headline             0
short_description    0
date                 0
category             0
dtype: int64
['PARENTING' 'THE WORLDPOST' 'PARENTING' ... 'PARENTING' 'PARENTING'
 'PARENTING']
               precision    recall  f1-score   support

    PARENTING       0.98      0.99      0.98      1189
THE WORLDPOST       0.96      0.95      0.95       394

     accuracy                           0.98      1583
    macro avg       0.97      0.97      0.97      1583
 weighted avg       0.98      0.98      0.98      1583



In [15]:
print(train_data.isnull().sum())
train_data = train_data.fillna("")
print(train_data.isnull().sum())

Unnamed: 0           0
headline             0
short_description    0
date                 0
category             0
text                 0
dtype: int64
Unnamed: 0           0
headline             0
short_description    0
date                 0
category             0
text                 0
dtype: int64
